In [1]:
"""
# 2 pytorch model
깊은 신경망을 이용한 classification 문제 해결
"""

'\n# 2 pytorch model\n깊은 신경망을 이용한 classification 문제 해결\n'

In [2]:
"""
# 첫번째 레이어의 형태 (Convolutional Layer)
합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
MaxPooling(kernel_size=2, stride=2)

# 두번째 레이어의 형태 (Convolutional Layer)
합성곱(in_channel=32, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
MaxPooling(kernel_size=2, stride=2)

# 세번째 레이어의 형태 (Convolutional Layer)
합성곱(in_channel=64, out_channel=128, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
MaxPooling(kernel_size=2, stride=2, padding=1)

# 네번째 레이어 : 전결합층(FC layer)
특성맵을 펼치는 역할
FC(Fully connected) layer + 활성화 함수(ReLU)

# 다섯번째 레이어: 전결합층(FC layer)
FC(Fully connected) layer + 활성화 함수(Softmax)
(마지막 노드는 10개) -> 0~9까지의 정답을 가져야 하기 때문에 노드를 10개를 설정
"""

'\n# 첫번째 레이어의 형태 (Convolutional Layer)\n합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)\nMaxPooling(kernel_size=2, stride=2)\n\n# 두번째 레이어의 형태 (Convolutional Layer)\n합성곱(in_channel=1, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)\nMaxPooling(kernel_size=2, stride=2)\n\n# 세번째 레이어의 형태 (Convolutional Layer)\n합성곱(in_channel=1, out_channel=128, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)\nMaxPooling(kernel_size=2, stride=2, padding=1)\n\n# 네번째 레이어 : 전결합층(FC layer)\n특성맵을 펼치는 역할\nFC(Fully connected) layer + 활성화 함수(ReLU)\n\n# 다섯번째 레이어: 전결합층(FC layer)\nFC(Fully connected) layer + 활성화 함수(Softmax)\n(마지막 노드는 10개) -> 0~9까지의 정답을 가져야 하기 때문에 노드를 10개를 설정\n'

In [3]:
# 실습 진행에 필요한 라이브러리 import
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [4]:
# GPU 사용을 위한 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드를 고정
torch.manual_seed(777)

# 위의 고정 랜덤 시드는 GPU가 사용 가능 할 경우에 고정됨
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
# 학습에 필요한 파라미터 설정하기
learning_rate = 0.001
training_epochs = 100
batch_size = 100

In [6]:
# torchvision 데이터로드의 데이터셋을 정의
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [7]:
# 배치 크기를 지정하는 과정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

# 60,000개의 데이터가 batch_size 100의 크기로 진행되기 때문에 total batch는 600이 됨

In [8]:
# 클래스를 이용하여 모델을 설계하기
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        # 첫번째 레이어
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # 두번째 레이어
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # 세번째 레이어
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64, 128,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        # 네번째 레이어
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1,
                                         torch.nn.ReLU(),
                                         torch.nn.Dropout(p=1 - self.keep_prob)) # 드롭아웃 : 과적합 방지
        
        # 다섯번째 레이어
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    # 순전파 전달을 위한 함수 forward 선언
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # 텐서를 펼치는 역할 : Flatten
        out = self.layer4(out)
        out = self.fc2(out)
        
        return out

In [9]:
# CNN 모델을 정의
model = CNN().to(device)

In [10]:
# 비용함수와 옵티마이저 선언
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# 총 배치의 수 확인하기(파라미터 조정하기)
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

# 사용자가 지정한 배치의 수에 따라 총 배치의 수가 달라짐을 볼 수 있음
# 배치 횟수 변경, 에폭 횟수 변경, 옵티마이저 변경, 손실함수 변경(비용함수) 등을
# 여러 방법으로 변경해보며 어떤 환경이 최적이었는지 판단하는 과정을 거쳐, 사용에 있어서 최적의 모델을 만들어 냄

총 배치의 수 : 600


In [15]:
# 학습 진행
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.0520880297
[Epoch:    2] cost = 0.0373753347
[Epoch:    3] cost = 0.0291581377
[Epoch:    4] cost = 0.024159167
[Epoch:    5] cost = 0.0190590918
[Epoch:    6] cost = 0.0166242737
[Epoch:    7] cost = 0.0149003444
[Epoch:    8] cost = 0.0119782947
[Epoch:    9] cost = 0.0119551318
[Epoch:   10] cost = 0.0109440945
[Epoch:   11] cost = 0.00998318754
[Epoch:   12] cost = 0.00926795416
[Epoch:   13] cost = 0.00769248838
[Epoch:   14] cost = 0.00677237147
[Epoch:   15] cost = 0.00631714566
[Epoch:   16] cost = 0.00807369594
[Epoch:   17] cost = 0.00612992607
[Epoch:   18] cost = 0.00635896716
[Epoch:   19] cost = 0.00416793255
[Epoch:   20] cost = 0.00606178399
[Epoch:   21] cost = 0.00526609458
[Epoch:   22] cost = 0.0049467748
[Epoch:   23] cost = 0.00411565835
[Epoch:   24] cost = 0.00376606663
[Epoch:   25] cost = 0.00679196091
[Epoch:   26] cost = 0.00490249228
[Epoch:   27] cost = 0.00502852257
[Epoch:   28] cost = 0.00312327174
[Epoch:   29] cost = 0.003257532

In [19]:
# 모델 테스트
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    
    print("Accuracy : ", accuracy.item())

Accuracy :  0.9560999870300293
